<a href="https://colab.research.google.com/github/TOKTAK007/Alexnet-with-TOKTAK/blob/main/Alexnet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Set environment
import sys
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import random

ในการสร้างโมเดล Machine Learning หรือ Deep Learning ส่วนใหญ่จะใช้ข้อมูลสำหรับการสร้างโมเดลเป็นชุดเดียว แต่ในขั้นตอนการสร้างโมเดลนั้น เราจะแบ่งชุดข้อมูลนั้นออกเป็น 2 ชุด คือ Training set และ Validation set

Training set คือชุดข้อมูลที่ใช้สำหรับการสร้างโมเดล โมเดลจะถูก train ด้วยชุดข้อมูลนี้โดยใช้ algorithm หรือโมเดลที่เราได้เลือกไว้ เพื่อให้โมเดลเรียนรู้วิธีการแยกแยะระหว่างแบบฝึกหัด (training data) ที่ต่างกันอย่างไร

Validation set คือชุดข้อมูลที่เอาไว้ใช้ตรวจสอบประสิทธิภาพของโมเดลที่ได้ train แล้ว โดย Validation set จะไม่ถูกนำไป train โมเดลแต่จะถูกนำไปใช้ในการประเมินว่าโมเดลที่เราได้สร้างขึ้นมามีความแม่นยำในการทำนายข้อมูลดีหรือไม่ ด้วยการเทียบผลการทำนายกับคำตอบจริงของ Validation set

สรุปกันย่อ ๆ คือ Training set เอาไว้สำหรับ train model และ Validation set เอาไว้ใช้ในการประเมินคุณภาพของโมเดลที่ได้ train ขึ้นมาด้วยการทำนายข้อมูลที่ไม่เคยเห็น (unseen data) ก่อนที่จะนำโมเดลไปใช้งานจริง


---



img_dim เป็นตัวแปรที่ใช้เก็บค่าขนาดของรูปภาพ โดยในที่นี้ img_dim = 32 หมายถึงขนาดของรูปภาพเป็น 32x32 pixel (หรือกว้าง 32 พิกเซล ยาว 32 พิกเซล) โดยรูปภาพนี้มี 3 ช่องสี (RGB) ดังนั้นขนาดของรูปภาพทั้งหมดคือ 32x32x3 (หรือ 3072)



---



In [ ]:
#Step 0: Predefined Parameters.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

datasets_path = '/Users/user/Desktop/test/input_data/datasets'
train_size = 0.7; val_size = 0.2; test_size = 0.1
seed = 42
batch_size = 32
model_name = 'TOKTAK'

checkpoint_path = '/Users/user/Desktop/test/input_data/train_model/' + model_name + '.ckpt'
output_dim = 2
image_dim_1 = 32; image_dim_2 = 32
epochs = 20
image2plot = 10

In [ ]:
#Step 1: Splitting the Dataset and Viewing Images. 

# class_names = os.listdir(datasets_path)
class_names = [f.name for f in os.scandir(datasets_path) if f.is_dir() and not f.name.startswith('.')]
print('class names: ', class_names)
num_class = len(class_names)
image_files=glob.glob(datasets_path + '/*/*.png', recursive=True)

print('total images in: ', datasets_path, ' is ', len(image_files))

idx_to_class = {i:j for i, j in enumerate(class_names)}
class_to_idx = {value:key for key, value in idx_to_class.items()}
train_idx, test_idx, val_idx = random_split(image_files, [train_size, val_size, test_size])
train_list=[image_files[i] for i in train_idx.indices]
val_list=[image_files[i] for i in test_idx.indices]
test_list=[image_files[i] for i in val_idx.indices]

print('number of training images: ', len(train_list),
	'\nnumber of val images: ', len(val_list),
	'\nnumber of test images: ', len(test_list))

def view_images(train_list, num_class):
	num_rows = 5; num_cols = 5
	num_images = num_rows * num_cols
	fig, axes = plt.subplots(num_rows, num_cols)
	displayed_classes = set()
	while len(displayed_classes) < num_class:
		random_images = random.sample(train_list, num_images)
		for i, ax in enumerate(axes.flatten()):
			img_path = random_images[i]
			img = Image.open(img_path)
			class_name = img_path.split('/')[-2]
			ax.imshow(img)
			ax.set_title(f'{class_name}')
			ax.axis('off')
			displayed_classes.add(class_name)
	plt.tight_layout()
	plt.show()

#view_images(train_list, num_class)

โค้ด train_data_flat = train_data.reshape(train_data.shape[0], -1) เป็นการแปลงข้อมูลใน train_data จากรูปแบบ 3 มิติ (สูง, กว้าง, ลึก) เป็นข้อมูลแบบ 2 มิติ (จำนวนตัวอย่าง, จำนวนคุณสมบัติ) 

โดยใช้ฟังก์ชัน reshape ซึ่งเป็นฟังก์ชันของ NumPy

การใช้ -1 ใน parameter ที่สองของ reshape จะหมายถึงการเปลี่ยนขนาดแกนที่สองของ train_data ให้เป็นขนาดที่เหมาะสมกับขนาดของข้อมูลที่กำลังจะเปลี่ยนแปลง (ในที่นี้คือ train_data.shape[0]) โดย reshape จะคำนวณขนาดแกนที่สองอัตโนมัติโดยให้เหมาะสมกับขนาดของข้อมูลที่กำลังจะเปลี่ยนแปลง

ดังนั้น train_data_flat จะเป็น numpy array ของข้อมูลแบบ 2 มิติที่มีจำนวนตัวอย่างเท่ากับ train_data 

และจำนวนคุณสมบัติเท่ากับจำนวนพิกเซลของรูปภาพ โดยแต่ละตัวอย่างจะเป็นเวกเตอร์ของค่าพิกเซลทั้งหมดในรูปภาพที่มีขนาดเท่ากับจำนวนคุณสมบัติ


---

In [ ]:
#Step 2: Data Preprocessing

from sklearn.preprocessing import StandardScaler
from skimage import io
import numpy as np

def means_std(train_list):
	train_data = []
	for img_path in train_list:
		img = io.imread(img_path)
		img = img / 255.0  # limit value to be between 0 and 1
		train_data.append(img)
	train_data = np.array(train_data)
	train_data = np.transpose(train_data, (0, 3, 1, 2))  # fit PyTorch format
	train_data_flat = train_data.reshape(train_data.shape[0], -1) #comment อยู่ด้านบน
	scaler = StandardScaler()
	scaler.fit(train_data_flat)
	mean = scaler.mean_.reshape(3, -1).mean(axis=1)
	std = scaler.scale_.reshape(3, -1).std(axis=1)
	print('mean: ', mean)
	print('standard deviation: ', std)
	return mean, std

from torchvision import transforms

class SatelliteDataset():
	def __init__(self, image_paths, class_to_idx, transform=None):
		self.image_paths = image_paths
		self.class_to_idx = class_to_idx
		self.transform = transform

	def __len__(self):
		return len(self.image_paths)

	def __getitem__(self, idx):
		image_filepath = self.image_paths[idx]
		image = Image.open(image_filepath)
		label = image_filepath.split('/')[-2]
		label = self.class_to_idx[label]
		if self.transform is not None:
			image = self.transform(image)
		return image, label

# mean, std = means_std(train_list)

mean = [0.27140397, 0.28222303, 0.23752819]
std = [0.00029074, 0.00028769, 0.00028466]

flip = transforms.RandomHorizontalFlip()
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean, std)

transform_train = transforms.Compose([flip, to_tensor, normalize])
transform_val = transforms.Compose([to_tensor, normalize])
transform_test = transforms.Compose([to_tensor, normalize])

train_dataset = SatelliteDataset(train_list, class_to_idx, transform_train)
val_dataset = SatelliteDataset(val_list, class_to_idx, transform_val)
test_dataset = SatelliteDataset(test_list, class_to_idx, transform_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size,
    shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size,
    shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size,
    shuffle=False, drop_last=False)

# view_images(train_loader, num_class, class_names, mean=mean, std=std)

คำสั่ง train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True) จะสร้าง DataLoader ของชุดข้อมูลการฝึก (training dataset) 

โดยใช้ข้อมูลจาก train_dataset ซึ่งสร้างจากขั้นตอนการแปลงรูปแบบของข้อมูลและการทำสเกล และกำหนด batch_size ของข้อมูลในแต่ละรอบการฝึก โดย shuffle=True จะทำให้ข้อมูลในแต่ละ batch ถูกสลับลำดับสุ่มใหม่เพื่อลดการเรียนรู้ลำดับของข้อมูลและป้องกันการเรียนรู้ pattern ที่ไม่ถูกต้องจากลำดับของข้อมูลเดิม และ drop_last=True จะทำให้ batch สุดท้ายที่ไม่เพียงพอสำหรับ batch_size จะถูกลบทิ้งเพื่อไม่ให้เกิดข้อผิดพลาดในการฝึก


---



scaler.fit(train_data_flat) คือการฝึกหรือปรับแต่ง Scaler ด้วยข้อมูล training ที่ได้จากการแปลงข้อมูลรูปภาพเป็นเวกเตอร์ของค่าพิกเซลที่มีขนาดเท่ากับจำนวนคุณสมบัติ โดย Scaler เป็นเครื่องมือที่ใช้ในการปรับแต่งข้อมูลให้เหมาะสมกับการฝึกโมเดล โดยมักนิยมใช้ StandardScaler หรือ MinMaxScaler ในการปรับแต่งข้อมูล ซึ่ง StandardScaler จะปรับให้ข้อมูลมี mean ศูนย์และ standard deviation เท่ากับหนึ่ง ส่วน MinMaxScaler จะปรับให้ข้อมูลอยู่ในช่วง [0, 1]

การทำ scaler.fit(train_data_flat) จะคำนวณค่า mean และ standard deviation หรือ min-max value ของแต่ละคุณสมบัติในข้อมูล training และเก็บไว้เพื่อใช้ในการปรับแต่งข้อมูลในขั้นตอนถัดไป เช่นการปรับแต่งข้อมูลในชุดข้อมูล validation หรือ test ก่อนนำข้อมูลมาฝึกโมเดล หรือในการใช้โมเดลจริง ๆ ในการทำนายข้อมูลจริง ๆ ดังนั้นการปรับแต่งข้อมูลด้วย Scaler เป็นส่วนสำคัญของการประมวลผลและการฝึกโมเดลที่มีประสิทธิภาพสูง

---



คำสั่ง mean = scaler.mean_.reshape(3, -1).mean(axis=1) เป็นการหาค่าเฉลี่ยของ mean ของแต่ละ feature โดยใช้ Scaler ที่เคยถูกปรับแต่งด้วยข้อมูล training ที่ผ่านการแปลงรูปแบบการสเกลข้อมูลใหม่แล้ว 

โดยการเรียกใช้ scaler.mean_ จะเป็นการเรียกค่า mean ของแต่ละ feature ที่ถูกปรับแต่งไว้ และ reshape(3, -1) จะเป็นการแบ่งค่า mean นี้ออกเป็น 3 ส่วน โดยแบ่งแต่ละ feature ให้อยู่ในแถวแยกกัน ซึ่งจะเห็นได้จากการกำหนดให้มี 3 แถว (rows) และ -1 ใน column ที่ไม่ระบุขนาด จะทำให้คำนวณค่า mean ของแต่ละ feature ในแต่ละแถว จากนั้นใช้ mean(axis=1) เพื่อหาค่าเฉลี่ยของแต่ละแถว ซึ่งจะได้เป็นเวกเตอร์ของค่าเฉลี่ย mean ของแต่ละ feature ในชุดข้อมูล training นั้น ๆ


---



In [ ]:
#Step 3: Model Initialization and Setup. 

class AlexNet(nn.Module):
	def __init__(self, output_dim):
		super().__init__()
		self.features = nn.Sequential(
			nn.Conv2d(3, 64, 3, 2, 1),  # in_channels(3RGB), out_channels(64 filter), kernel_size, stride, padding
			nn.MaxPool2d(2),  # kernel_size
			nn.ReLU(inplace=True),
			nn.Conv2d(64, 192, 3, padding=1),
			nn.MaxPool2d(2),
			nn.ReLU(inplace=True),
			nn.Conv2d(192, 384, 3, padding=1),
			nn.ReLU(inplace=True),
			nn.Conv2d(384, 256, 3, padding=1),
			nn.ReLU(inplace=True),
			nn.Conv2d(256, 256, 3, padding=1),
			nn.MaxPool2d(2),
			nn.ReLU(inplace=True)
		)
		self.classifier = nn.Sequential(
			nn.Dropout(0.5),
			nn.Linear(256 * 2 * 2, 4096),
			nn.ReLU(inplace=True),
			nn.Dropout(0.5),
			nn.Linear(4096, 4096),
			nn.ReLU(inplace=True),
			nn.Linear(4096, output_dim),
		)

	def forward(self, x):
		x = self.features(x)
		h = x.view(x.shape[0], -1)
		x = self.classifier(h)
		return x, h

from torchsummary import summary

model = AlexNet(output_dim)

summary(model, (3, image_dim_1, image_dim_2))
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

self.classifier เป็นชุดของเลเยอร์ (layer) ที่ใช้ในการประมวลผลขั้นต่อไปหลังจากการทำ feature extraction ด้วยชุดเลเยอร์ที่อยู่ใน self.features ที่กล่าวไปก่อนหน้านี้


nn.Dropout(0.5) เป็นการใช้ dropout ที่จะทำให้บางโหนดในชุดข้อมูลถูกลบออกจากการคำนวณทุกครั้งที่ทำการสุ่มสลับชุดข้อมูล เพื่อป้องกัน overfitting


nn.Linear(256 * 2 * 2, 4096) เป็นชั้นเลเยอร์เชิงเส้น (linear layer) ที่มีจำนวน output units เท่ากับ 4096 โดย input units จะมีขนาด 256 * 2 * 2 (256 คือจำนวน filters ที่ใช้ในชุดเลเยอร์ self.features และ 2x2 คือขนาดของ feature map หลังจากที่ผ่านชุดเลเยอร์ Convolutional และ Max Pooling)
nn.ReLU(inplace=True) เป็นการใช้ ReLU activation function ในการ activate output units จากชั้นเลเยอร์ก่อนหน้า
nn.Dropout(0.5) เช่นเดียวกับข้างบน
nn.Linear(4096, 4096) เป็นชั้นเลเยอร์เชิงเส้นที่มี input units เท่ากับ 4096 (จำนวน output units จากชั้นก่อนหน้า) 


และ output units เท่ากับ 4096
nn.ReLU(inplace=True) เช่นเดียวกับข้างบน
nn.Linear(4096, output_dim) เป็นชั้นเลเยอร์เชิงเส้นสุดท้ายที่จะคำนวณ output ของโมเดล โดย output units จะมีจำนวนเท่ากับ output_dim ซึ่งเป็นจำนวน class ของปัญหาที่ต้องการจะทำนาย


---






---
ฟังก์ชั่น forward ใช้สำหรับ forward pass ของโมเดล โดยทำการทำภาพผ่านชั้น self.features ซึ่งเป็นชั้นของโมเดล AlexNet ที่ใช้สำหรับการคัดแยกลักษณะของภาพ (feature extraction) 


จากนั้นแปลง output จากชั้น self.features เป็น vector แบบ 1 มิติ โดยการใช้ view แล้วนำไปผ่านชั้น self.classifier ซึ่งเป็นชั้น Fully Connected Layer เพื่อทำการคาดคะเนผลลัพธ์ของแต่ละ class จาก vector นั้น โดย output จะประกอบด้วยค่าที่คาดคะเนได้และ feature vector นั้นๆ ที่ได้จาก self.features


---






---
ฟังก์ชั่น calculate_accuracy ใช้สำหรับคำนวณความแม่นยำของโมเดล โดยรับ input สองตัวคือ y_pred และ y ที่เป็น tensor ที่มีขนาดเท่ากัน ซึ่งตัวแปร y_pred จะเก็บผลลัพธ์ที่ได้จากโมเดล และตัวแปร y จะเก็บ label จริงๆของข้อมูลตัวอย่าง


ฟังก์ชั่นจะทำการคำนวณว่าโมเดลทำนายถูกต้องกี่ตัวด้วยการหาค่า top_pred ซึ่งจะเป็นตัวเลขที่เก็บค่า label ที่โมเดลคาดว่าจะเป็น label จริง จากนั้นจึงนำ top_pred มาเปรียบเทียบกับ y เพื่อหาว่าโมเดลทำนายถูกหรือไม่ ด้วยการใช้ eq และ sum ในการหาจำนวนของ label ที่โมเดลทำนายถูกต้อง และหลังจากนั้นจึงคำนวณค่าความแม่นยำโดยหารจำนวน label ทั้งหมดของข้อมูลตัวอย่าง และคืนค่าความแม่นยำออกมา


---





In [ ]:
#Step 4: Training the Model.
from tqdm import tqdm

def calculate_accuracy(y_pred, y):
	top_pred = y_pred.argmax(1, keepdim=True)
	correct = top_pred.eq(y.view_as(top_pred)).sum()
	acc = correct.float() / y.shape[0]
	return acc

def train(model, iterator, optimizer, criterion, device):
	epoch_loss = 0; epoch_acc = 0
	model.train()
	for (x, y) in tqdm(iterator, desc='Training', leave=False):
		x = x.to(device)
		y = y.to(device)
		optimizer.zero_grad()
		y_pred, _ = model(x)
		loss = criterion(y_pred, y)
		acc = calculate_accuracy(y_pred, y)
		loss.backward()
		optimizer.step()
		epoch_loss += loss.item()
		epoch_acc += acc.item()
	return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion, device):
	epoch_loss = 0; epoch_acc = 0
	model.eval()
	with torch.no_grad():
		for (x, y) in tqdm(iterator, desc='Evaluating', leave=False):
			x = x.to(device)
			y = y.to(device)
			y_pred, _ = model(x)
			loss = criterion(y_pred, y)
			acc = calculate_accuracy(y_pred, y)
			epoch_loss += loss.item()
			epoch_acc += acc.item()
	return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
	elapsed_time = end_time - start_time
	elapsed_mins = int(elapsed_time / 60)
	elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
	return elapsed_mins, elapsed_secs

def loss_history_plot(history_train, history_valid, model_name):
	axis_x = np.linspace(0, len(history_train), len(history_train))
	plt.plot(axis_x, history_train, linestyle='solid',
			 color='red', linewidth=1, marker='o', ms=5, label='train')
	plt.plot(axis_x, history_valid, linestyle='solid',
			 color='blue', linewidth=1, marker='o', ms=5, label='valid')
	plt.xlabel('epoch')
	plt.ylabel('loss')
	plt.legend(['train', 'valid'])
	plt.title(model_name + ': ' + 'Accuracy', fontweight='bold')
	# plt.savefig('data_out/' + 'resnet' + '.svg', format='svg', bbox_inches='tight', transparent=True, pad_inches=0)
	plt.show()


history_train_loss = []
history_valid_loss = []
best_valid_loss = float('inf')
for epoch in range(epochs):
    start_time = time.monotonic()
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, val_loader, criterion, device)
    history_train_loss.append(train_loss)
    history_valid_loss.append(valid_loss)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), checkpoint_path)
    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
loss_history_plot(history_train_loss, history_valid_loss, model_name)




---
ค้ดบรรทัดนี้จะกำหนดค่าตั้งต้นให้กับตัวแปร best_valid_loss ด้วยค่า infinity (ได้มาจากฟังก์ชัน float() ที่ไม่มีการส่งค่าอะไรเข้าไป) ซึ่งจะถูกใช้ในการเก็บค่า loss ที่ดีที่สุดของ 

validation set ในแต่ละ epoch ถ้าหากค่า loss ของ validation set ใน epoch ปัจจุบันน้อยกว่าค่า loss ของ validation set ที่ดีที่สุดที่เคยเก็บไว้ 


---


(best_valid_loss) จะทำการอัพเดท best_valid_loss ด้วยค่า loss ของ validation set ใน epoch ปัจจุบัน และบันทึกโมเดลที่ดีที่สุดไว้ในไฟล์ checkpoint_path


---






---


ฟังก์ชัน train เป็นฟังก์ชันสำหรับการฝึก (training) โมเดลของเรา โดยรับพารามิเตอร์ดังนี้

model: โมเดลที่จะฝึก

iterator: ตัวเลข (iterator) ที่จะใช้ในการวน loop ในการดึงข้อมูล (data) จาก dataloader โดยในแต่ละ iteration จะส่งคืน tensor ของ feature และ label
optimizer: ออบเจกต์ Optimizer ที่ใช้ในการปรับพารามิเตอร์ของโมเดล

criterion: ค่าความผิดพลาดที่ใช้ในการคำนวณ 

gradient และ loss

device: ส่วนของ hardware (CPU หรือ GPU) ที่ใช้ในการฝึกโมเดล

ฟังก์ชันจะส่งคืนค่าเฉลี่ยของ loss และ accuracy ของโมเดลที่ถูกคำนวณจากข้อมูลทั้งหมดใน iterator โดยในแต่ละ iteration จะทำการ forward ข้อมูลผ่านโมเดล คำนวณ loss และ accuracy จากนั้นทำการ backward และปรับค่าพารามิเตอร์ใน optimizer ตามที่ได้คำนวณไว้ และสะสมค่า loss และ accuracy ในแต่ละ iteration โดยใช้เทคนิค gradient descent ในการปรับพารามิเตอร์ของโมเดล สุดท้ายฟังก์ชันจะส่งคืนค่าเฉลี่ยของ loss และ accuracy ที่ถูกคำนวณจากข้อมูลทั้งหมดใน iterator


---





---
tqdm เป็นโมดูลของ Python ที่ใช้สร้าง progress bar ในการแสดงผลการทำงานของ loop หรือ iterator ที่กำลังทำงานอยู่ เพื่อให้ผู้ใช้งานสามารถเห็นความคืบหน้าได้ง่ายขึ้น 

โดยจะแสดงเป็นเส้นตารางด้วยสีพื้นหลังเขียวที่แสดงถึงความคืบหน้าของการทำงาน สามารถติดตั้งได้ผ่าน pip หรือ conda และใช้งานได้ง่ายและสะดวก


---






---

ฟังก์ชัน loss_history_plot นี้ใช้สำหรับพล็อตกราฟแสดงค่า loss ของการเทรนโมเดลในแต่ละ epoch ของชุดข้อมูล train และ validation โดยรับ parameter ดังนี้


history_train: คือ list ที่เก็บค่า loss ของการเทรนโมเดลในแต่ละ epoch ของชุดข้อมูล train
history_valid: คือ list ที่เก็บค่า loss ของการเทรนโมเดลในแต่ละ epoch ของชุดข้อมูล validation

model_name: ชื่อของโมเดลที่ใช้เทรน
ฟังก์ชันนี้จะสร้างกราฟแสดงค่า loss ของการเทรนโมเดลในแต่ละ epoch ของชุดข้อมูล train และ validation โดยใช้ matplotlib.pyplot ในการพล็อตกราฟ และแสดงผลผ่านทาง plt.show()



---





---
โค้ดนี้เป็นการ train โมเดล neural network โดยใช้ฟังก์ชัน train และ evaluate ซึ่งรับอินพุตเป็นโมเดลที่ต้องการ train, ชุดข้อมูลที่ใช้ train (train_loader), optimizer, criterion และ device ที่ใช้ในการ train โมเดล โดยจะวนลูปผ่านจำนวน epochs ที่กำหนดไว้ 


และเก็บค่า train loss และ validation loss ไว้ในตัวแปร history_train_loss และ history_valid_loss ตามลำดับ ในแต่ละ epoch จะทำการประมวลผล train loss, train accuracy, validation loss และ validation accuracy แล้วแสดงผลทางหน้าจอ และบันทึกโมเดลที่ให้ผลลัพธ์ validation loss ที่ดีที่สุดไว้ในไฟล์ที่กำหนด 


ท้ายสุดจะทำการพล็อตกราฟ loss ของ train และ validation ด้วยฟังก์ชัน loss_history_plot ซึ่งเป็นการพล็อตกราฟแสดงการเปลี่ยนแปลง loss ของ train และ validation ตามจำนวน epoch ที่ train โมเดล


---




In [ ]:
#Step 5: Plotting Confusion Matrix.
from sklearn import metrics

def plot_confusion_matrix_CIFAR10(labels, pred_labels, classes):
	fig = plt.figure()
	ax = fig.add_subplot(1, 1, 1)
	cm = metrics.confusion_matrix(labels, pred_labels)
	cm = metrics.ConfusionMatrixDisplay(cm, display_labels=classes)
	cm.plot(values_format='d', cmap='Greens', ax=ax)
	# plt.savefig('data_out/' + 'CM_resnet' + '.svg', format='svg', bbox_inches='tight', transparent=True, pad_inches=0)
	plt.show()

import torch.nn.functional as TF

def get_predictions(model, iterator, device):
	model.eval()
	images = []; labels = []; probs = []
	with torch.no_grad():
		for (x, y) in iterator:
			x = x.to(device)
			y_pred, _ = model(x)
			y_prob = TF.softmax(y_pred, dim=-1)
			images.append(x.cpu())
			labels.append(y.cpu())
			probs.append(y_prob.cpu())
	images = torch.cat(images, dim=0)
	labels = torch.cat(labels, dim=0)
	probs = torch.cat(probs, dim=0)
	return images, labels, probs

model.load_state_dict(torch.load(checkpoint_path))
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
images, labels, probs = get_predictions(model, test_loader, device)
pred_labels = torch.argmax(probs, 1)
plot_confusion_matrix_CIFAR10(labels, pred_labels, class_names)